In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset_path = "creditcard.csv"
df = pd.read_csv(dataset_path)
df.info

In [ ]:
df_np = df.to_numpy()

X_features, y_target = df_np[:, :-1].astype(np.float64), df_np[:, -1].astype(np.int16)
X_features = np.hstack((np.ones((X_features.shape[0], 1)), X_features))
X_features.shape, y_target.shape

In [ ]:
# Chuyển đổi y_target thành ma trận nhị phân
n_classes = np.unique(y_target).shape[0]
n_samples = y_target.shape[0]

y_encoded = np.zeros((n_samples, n_classes))
y_encoded[np.arange(n_samples), y_target] = 1
y_encoded.shape

In [ ]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X_features,
    y_encoded,
    test_size=val_size,
    random_state=random_state,
    shuffle=is_shuffle,
)

X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle
)

X_train.shape, X_val.shape, X_test.shape,

In [ ]:
normalizer = StandardScaler()
X_train[:, 1:] = normalizer.fit_transform(X_train[:, 1:])
X_val[:, 1:] = normalizer.transform(X_val[:, 1:])
X_test[:, 1:] = normalizer.transform(X_test[:, 1:])

In [ ]:
def softmax(z):
    exp_z = np.exp(z)
    # Chia giá trị exp_z cho tổng của mỗi hàng để chuẩn hóa về [0, 1]
    # exp_z.sum(axis=1)[:, None] chuyển đổi từ (n_samples,) thành (n_samples, 1)
    # để có thể broadcast với exp_z
    return exp_z / exp_z.sum(axis=1)[:, None]


def predict(X, theta):
    return softmax(np.dot(X, theta))


def compute_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred), axis=1))


def compute_gradient(X, y_true, y_pred):
    return np.dot(X.T, (y_pred - y_true)) / X.shape[0]


def update_theta(theta, gradient, learning_rate):
    return theta - learning_rate * gradient


def compute_accuracy(y_true, y_pred):
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_true_labels = np.argmax(y_true, axis=1)
    return np.mean(y_pred_labels == y_true_labels)

In [ ]:
lr = 0.01
epochs = 20
batch_size = 1024
n_features = X_train.shape[1]

print(f"Number of features: {n_features}")
print(f"Number of classes: {n_classes}")

np.random.seed(random_state)
theta = np.random.uniform(size=(n_features, n_classes))

train_accs = []
val_accs = []
train_losses = []
val_losses = []

for epoch in range(epochs):
    train_batch_losses = []
    train_batch_accs = []
    val_batch_losses = []
    val_batch_accs = []
    for i in range(0, X_train.shape[0], batch_size):
        X_batch = X_train[i : i + batch_size]
        y_batch = y_train[i : i + batch_size]
        # train
        y_pred = predict(X_batch, theta)
        loss = compute_loss(y_batch, y_pred)
        gradient = compute_gradient(X_batch, y_batch, y_pred)
        theta = update_theta(theta, gradient, lr)

        train_batch_losses.append(loss)
        train_batch_accs.append(compute_accuracy(y_batch, y_pred))
        # validation
        y_val_pred = predict(X_val, theta)
        val_loss = compute_loss(y_val, y_val_pred)
        val_acc = compute_accuracy(y_val, y_val_pred)
        val_batch_losses.append(val_loss)
        val_batch_accs.append(val_acc)

    train_losses.append(np.mean(train_batch_losses))
    train_accs.append(np.mean(train_batch_accs))
    val_losses.append(np.mean(val_batch_losses))
    val_accs.append(np.mean(val_batch_accs))
    print(
        f"Epoch {epoch + 1}/{epochs} - "
        f"Train Loss: {train_losses[-1]:.4f}, "
        f"Train Accuracy: {train_accs[-1]:.4f}, "
        f"Validation Loss: {val_losses[-1]:.4f}, "
        f"Validation Accuracy: {val_accs[-1]:.4f}"
    )

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 10))

ax[0, 0].plot(train_losses)
ax[0, 0].set(xlabel="Epoch", ylabel="Loss")
ax[0, 0].set_title("Training Loss")

ax[0, 1].plot(val_losses, color="orange")
ax[0, 1].set(xlabel="Epoch", ylabel="Loss")
ax[0, 1].set_title("Validation Loss")

ax[1, 0].plot(train_accs)
ax[1, 0].set(xlabel="Epoch", ylabel="Accuracy")
ax[1, 0].set_title("Training Accuracy")

ax[1, 1].plot(val_accs, color="orange")
ax[1, 1].set(xlabel="Epoch", ylabel="Accuracy")
ax[1, 1].set_title("Validation Accuracy")

plt.show()

In [ ]:
val_set_acc = compute_accuracy(y_val, predict(X_val, theta))
test_set_acc = compute_accuracy(y_test, predict(X_test, theta))
print("Evaluation on validation and test set:")
print(f"Validation Accuracy: {val_set_acc}")
print(f"Test Accuracy: {test_set_acc}")